# Wind prediction - Second assignment

## Authors

David Moreno Maldonado 100441714     
Inés Fernández Campos 100443936

## 0. Preliminaries

In [1]:
# Import some libraries
import os
import numpy as np              
import pandas as pd
import matplotlib.pyplot as plt 

import sys
import time
import math

from sklearn.experimental import enable_iterative_imputer
from sklearn import preprocessing, impute, model_selection, metrics, neighbors, ensemble, feature_selection
from sklearn.pipeline import Pipeline
import optuna
import optuna.visualization as ov

os.getcwd()

'/home/fddcampos/Documents/uc3m/2_term/BDINTELLIGENCE/practicas_big_data_intelligence/assignment_2'

In [2]:
#MAIN PARAMETERS FOR THE ASSIGNMENT
budget = 20
random_state = 3
verbose = 0
n_jobs = 1

The "wind_pickle" file contains data in a binary format called "Pickle". Pickle data loads faster than text data.

In [3]:
data = pd.read_pickle('wind_pickle.pickle')

You can visualize the attributes in the dataset. Very important, the output attribute (i.e. the value to be predicted, **energy**, is the first attribute). **Steps** represents the hours in advance of the forecast. We will not use this variable here.

In [4]:
# The dataset contains 5937 instances and 556 attributes (including the outcome to be predicted)
print(data.shape)
#data.columns.values.tolist() 

(5937, 556)


In [5]:
#-1 for training, 0 for validation, 1 for testing
year_to_part = {
    2005: -1,
    2006: -1,
    2007: 0,
    2008: 0, 
    2009: 1,
    2010: 1
}
data['partition'] = data['year'].apply(lambda x: year_to_part[x])

We now remove the columns that cannot be used for training the models from the DataFrame

In [6]:
# Steps, month, day, hour, year should be removed, they cannot be used for training the models
to_remove = ['steps', 'month', 'year', 'day', 'hour']
for m in to_remove: data = data.drop(m, 1)

In [7]:
from numpy.random import randint

# we add na values at random
my_NIA = 100443936 + 100441714
np.random.seed(my_NIA)

how_many_nas = round(data.shape[0]*data.shape[1]*0.05)
print('Lets put '+str(how_many_nas)+' missing values \n')
x_locations = randint(0, data.shape[0], size=how_many_nas)
y_locations = randint(1, data.shape[1]-2, size=how_many_nas)

for i in range(len(x_locations)):
    data.iat[x_locations[i], y_locations[i]] = np.nan
    
data.to_pickle('wind_pickle_with_nan.pickle')

Lets put 163861 missing values 



From this point on, the file wind_pickle_with_nan should be used.

In [8]:
data = pd.read_pickle('wind_pickle_with_nan.pickle')
data.shape

(5937, 552)

## Input missing data

In [9]:
print(data.isnull().values.any())
input_cols = data.columns.difference(['energy', 'partition'])
x = data[input_cols]
#Iterative imputer (takes too long)
'''iter_imp = impute.IterativeImputer(random_state=random_state, 
                                   initial_strategy='median', 
                                   max_iter=3,
                                   verbose=verbose)
no_nan = iter_imp.fit_transform(x)'''

#KNN imputer(takes too long)
'''knn_imp = impute.KNNImputer(weights='distance')
no_nan = knn_imp.fit_transform(x)'''

#Simple imputer
simp_imp = impute.SimpleImputer(strategy='median',
                               verbose=2)
no_nan = simp_imp.fit_transform(x)

data[input_cols] = pd.DataFrame(data=no_nan)
print(data.isnull().values.any())

True
False


## Scaling

In [10]:
scaler = preprocessing.StandardScaler().fit(data[input_cols]) 
data[input_cols] = scaler.transform(data[input_cols])

## Data split
We are going to use train/test for model evaluation (outer) and train/validation for hyperparameter tuning (inner), as follows:     
1. Train partition: the first two years of data. Given that there are 6 years worth of data, we will use the first 2/6 of the instances for training.     
2. Validation partition: the second two years of data. 
3. Test partition: the remaining data    


In [11]:
#-1 for training, 0 for validation, 1 for testing
test = data[data['partition'] == 1]
train = data[data['partition'] == -1]
val = data[data['partition'] == 0]

del test['partition']
del train['partition']

y_test = test['energy']
x_test = test[test.columns.difference(['energy'])]

y_train = train['energy']
x_train = train[train.columns.difference(['energy'])]


y_val = val['energy']
x_val = val[train.columns.difference(['energy'])]

# 1. MODEL SELECTION AND HYPER-PARAMETER TUNING

In [13]:
#Dataframes with all the information of each model
summary = {
    'knn': pd.DataFrame(columns=['Time (sec)', 'Score (RMSE)', 'N. neighbors', 'Weights', 'P']),
    'random_forest': pd.DataFrame(columns=['Time (sec)', 'Score (RMSE)', 'Min. samples split', 'Criterion', 'Max. depth', 'N. estimators','Max. features']),
    'gradient_boosting': pd.DataFrame(columns=['Time (sec)', 'Score (RMSE)'])
}

## 1.1 KNN

### 1.1.1 Default hyper-parameters

In [15]:
np.random.seed(random_state)
knn_default = neighbors.KNeighborsRegressor()

start_time = time.time()
knn_default = knn_default.fit(x_train, y_train)
y_val_pred = knn_default.predict(x_val)
score = math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))
end_time = time.time()

summary['knn'] = summary['knn'].append(pd.Series({
    'Time (sec)': '{:.4f}'.format(end_time - start_time), 
    'Score (RMSE)': score, 
    'N. neighbors': 5, 
    'Weights': 'uniform', 
    'P': 2
    }, 
    name='default'))

### 1.1.2 Hyper-parameter tunning (OPTUNA)

In [16]:
min_n_neigbors = 1
max_n_neigbors = 16

In [17]:
np.random.seed(random_state)
def knn_objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', min_n_neigbors, max_n_neigbors)
    weights = trial.suggest_categorical('weights', ['uniform','distance'])
    p = trial.suggest_categorical('p', [1, 2])

    clf = neighbors.KNeighborsRegressor(
        n_neighbors=n_neighbors,
        weights=weights,
        p=p)
    
    clf = clf.fit(x_train, y_train)
    y_val_pred = clf.predict(x_val)
    return math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))

knn_optuna = optuna.create_study(direction='minimize')
start_time = time.time()
knn_optuna.optimize(knn_objective, n_trials=budget)
end_time = time.time()

summary['knn'] = summary['knn'].append(pd.Series({
    'Time (sec)': '{:.4f}'.format(end_time - start_time), 
    'Score (RMSE)': knn_optuna.best_value, 
    'N. neighbors': knn_optuna.best_params['n_neighbors'], 
    'Weights': knn_optuna.best_params['weights'], 
    'P': knn_optuna.best_params['p']
    }, 
    name='optuna'))

[I 2021-01-13 14:30:52,839] A new study created in memory with name: no-name-5643f019-8098-4f6e-a30e-e1b102ce75a1
[I 2021-01-13 14:30:55,885] Trial 0 finished with value: 461.50247231150104 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 461.50247231150104.
[I 2021-01-13 14:30:59,358] Trial 1 finished with value: 425.8532193925109 and parameters: {'n_neighbors': 12, 'weights': 'uniform', 'p': 1}. Best is trial 1 with value: 425.8532193925109.
[I 2021-01-13 14:31:02,432] Trial 2 finished with value: 446.53181088677667 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'p': 2}. Best is trial 1 with value: 425.8532193925109.
[I 2021-01-13 14:31:05,247] Trial 3 finished with value: 438.66704912167074 and parameters: {'n_neighbors': 8, 'weights': 'distance', 'p': 2}. Best is trial 1 with value: 425.8532193925109.
[I 2021-01-13 14:31:08,229] Trial 4 finished with value: 435.96737369410965 and parameters: {'n_neighbors': 16, 'weights': 'un

## 1.2 Random Forest

### 1.2.1 Default hyper-parameters

In [20]:
np.random.seed(random_state)
rf_default = ensemble.RandomForestRegressor(random_state=random_state, verbose=verbose, n_jobs=n_jobs)

start_time = time.time()
rf_default = rf_default.fit(x_train, y_train)
y_val_pred = rf_default.predict(x_val)
score =  math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))
end_time = time.time()

summary['random_forest'] = summary['random_forest'].append(pd.Series({
    'Time (sec)': '{:.4f}'.format(end_time - start_time), 
    'Score (RMSE)': score,
    'Min. samples split': 2, 
    'Criterion': 'mse', 
    'Max. depth': 'None',
    'N. estimators': 100,
    'Max. features': 1
    },
    name='default'))

### 1.2.2 Hyper-parameter tunning (OPTUNA)

In [21]:
min_max_depth = 2
max_max_depth = 50
min_n_estimators = 50
max_n_estimators = 200

In [22]:
np.random.seed(random_state)
def random_forest_objective(trial):
    min_samples_split = trial.suggest_uniform('min_samples_split', 0+sys.float_info.min, 1)
    criterion = trial.suggest_categorical('criterion', ['mse','mae'])
    max_depth = trial.suggest_int('max_depth', min_max_depth, max_max_depth)
    n_estimators = trial.suggest_int('n_estimators', min_n_estimators, max_n_estimators)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])

    clf = ensemble.RandomForestRegressor(
        random_state=random_state,
        min_samples_split=min_samples_split,
        criterion=criterion,
        max_depth=max_depth,
        n_estimators=n_estimators,
        max_features=max_features
        )
    clf = clf.fit(x_train, y_train)
    y_val_pred = clf.predict(x_val)
    return math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))

rf_optuna = optuna.create_study(direction='minimize')
start_time = time.time()
rf_optuna.optimize(random_forest_objective, n_trials=budget, n_jobs=n_jobs)
end_time = time.time()

summary['random_forest'] = summary['random_forest'].append(pd.Series({
    'Time (sec)': '{:.4f}'.format(end_time - start_time), 
    'Score (RMSE)': rf_optuna.best_value,
    'Min. samples split': rf_optuna.best_params['min_samples_split'], 
    'Criterion': rf_optuna.best_params['criterion'], 
    'Max. depth': rf_optuna.best_params['max_depth'],
    'N. estimators': rf_optuna.best_params['n_estimators'],
    'Max. features': rf_optuna.best_params['max_features']
    },
    name='optuna'))

[I 2021-01-13 14:40:46,203] A new study created in memory with name: no-name-2663b009-5998-4065-8d0e-10ee2f47a415
[I 2021-01-13 14:40:48,116] Trial 0 finished with value: 400.9598210339549 and parameters: {'min_samples_split': 0.2585716150647326, 'criterion': 'mse', 'max_depth': 21, 'n_estimators': 119, 'max_features': 'sqrt'}. Best is trial 0 with value: 400.9598210339549.
[I 2021-01-13 14:40:48,275] Trial 1 finished with value: 668.3836578998909 and parameters: {'min_samples_split': 0.7275958768930648, 'criterion': 'mse', 'max_depth': 19, 'n_estimators': 125, 'max_features': 'sqrt'}. Best is trial 0 with value: 400.9598210339549.
[I 2021-01-13 14:40:48,379] Trial 2 finished with value: 668.4502497483671 and parameters: {'min_samples_split': 0.9734866223139015, 'criterion': 'mse', 'max_depth': 19, 'n_estimators': 83, 'max_features': 'sqrt'}. Best is trial 0 with value: 400.9598210339549.
[I 2021-01-13 14:40:48,521] Trial 3 finished with value: 668.3890217234632 and parameters: {'min_s

## 1.3 Gradient Boosting

### 1.3.1 Default hyper-parameters

In [23]:
# implementation using sklearn
np.random.seed(random_state)
gb_sk_def = ensemble.GradientBoostingRegressor(random_state=random_state, verbose=verbose)

start_time = time.time()
gb_sk_def = gb_sk_def.fit(x_train, y_train)
y_val_pred = gb_sk_def.predict(x_val)
score =  math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))
end_time = time.time()

summary['gradient_boosting'] = summary['gradient_boosting'].append(pd.Series({
    'Time (sec)': '{:.4f}'.format(end_time - start_time), 
    'Score (RMSE)': score,
    'Learning rate': 0.1,
    'N. estimators': 100,
    'Criterion': 'friedman_mse', 
    'Min. samples split': 2, 
    'Min. samples leaf': 1,
    'Max. depth': 3,
    'Max. leaf nodes': 'None'
    },
    name='default'))

In [27]:
# implementation using xgboost
import xgboost as xgb

dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

gb_xgb_def = xgb.XGBRegressor(objective='reg:squarederror')

start_time = time.time()
gb_xgb_def = gb_xgb_def.fit(x_train, y_train)
y_val_pred = gb_xgb_def.predict(x_val)
score = math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))
end_time = time.time()

summary['gradient_boosting'] = summary['gradient_boosting'].append(pd.Series({
    'Time (sec)': '{:.4f}'.format(end_time - start_time), 
    'Score (RMSE)': score,
    'Learning rate': 0.3,
    'Max. depth': 6,
    'Max. leaf nodes': 0,
    'Gamma (min_split_loss)': 0,
    'Lambda': 1,
    'Alpha': 0,
    'N. estimators': gb_xgb_def.get_params()['n_estimators']
    },
    name='default_xgboost'))

### 1.3.2 Hyper-parameter tunning

In [28]:
#print(dir(dtrain))
#print('\n', dir(model))
min_max_leaf_nodes = 2
max_max_leaf_nodes = 20
min_min_samples_leaf = 1
max_min_samples_leaf = 10

In [ ]:
# hyperparam tuning for sklearn ensemble.GradientBoostingRegressor
np.random.seed(random_state)

def gradboosting_objective(trial):  
    gb_sk_opt = None
    short = False
    
    learning_rate = trial.suggest_uniform('learning_rate', 0+sys.float_info.min, 1)
    n_estimators = trial.suggest_int('n_estimators', min_n_estimators, max_n_estimators)
    min_samples_split = trial.suggest_uniform('min_samples_split', 0+sys.float_info.min, 1)
    max_depth = trial.suggest_int('max_depth', min_max_depth, max_max_depth)
        
    if short == False: # it will take a long time to run 
        criterion = trial.suggest_categorical('criterion', ['mse','friedman_mse'])
        min_samples_leaf = trial.suggest_int('min_samples_leaf',min_min_samples_leaf, max_min_samples_leaf)
        max_leaf_nodes = trial.suggest_int('max_leaf_nodes', min_max_leaf_nodes, max_max_leaf_nodes)
        
        gb_sk_opt = ensemble.GradientBoostingRegressor(learning_rate=learning_rate, 
                                                   n_estimators=n_estimators,
                                                   criterion=criterion,
                                                   min_samples_split=min_samples_split,
                                                   min_samples_leaf=min_samples_leaf,
                                                   max_depth=max_depth,
                                                   max_leaf_nodes=max_leaf_nodes,
                                                   random_state=random_state,
                                                   verbose=verbose)
    else:  # will take less time        
        gb_sk_opt = ensemble.GradientBoostingRegressor(learning_rate=learning_rate, 
                                                   n_estimators=n_estimators,
                                                   min_samples_split=min_samples_split,
                                                   max_depth=max_depth,
                                                   random_state=random_state,
                                                   verbose=verbose)
        
    gb_sk_opt = gb_sk_opt.fit(x_train, y_train)
    y_val_pred = gb_sk_opt.predict(x_val)
    
    return math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))

gb_optuna = optuna.create_study(direction='minimize')
start_time = time.time()
gb_optuna.optimize(gradboosting_objective, n_trials=budget)
end_time = time.time()

summary['gradient_boosting'] = summary['gradient_boosting'].append(pd.Series({
    'Time (sec)': '{:.4f}'.format(end_time - start_time), 
    'Score (RMSE)': gb_optuna.best_value,
    'Learning rate': gb_optuna.best_params['learning_rate'],
    'N. estimators': gb_optuna.best_params['n_estimators'],
    'Criterion': 'friedman_mse', 
    #Min. samples split': gb_optuna.best_params['min_samples_split'], 
    'Min. samples leaf': gb_optuna.best_params['min_samples_leaf'],
    'Min. samples leaf': 1,
    'Max. depth': gb_optuna.best_params['max_depth'],
    #Max. leaf nodes': 'None'
    'Max. leaf nodes': gb_optuna.best_params['max_leaf_nodes']
    },
    name='optuna_sklearn'))

In [30]:
def xgradboosting_objective(trial):  
    
    param = {
        'booster': 'gbtree',
        #'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        #'alpha': trial.suggest_uniform('alpha', 0+sys.float_info.min, 1),
        #'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'lambda': trial.suggest_uniform('lambda', 0.01, 0.5),
        'alpha': trial.suggest_uniform('alpha', 0.01, 0.5),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'eta': trial.suggest_uniform('eta', 0+sys.float_info.min, 1.0),
        'max_depth': trial.suggest_int('max_depth', min_max_depth, max_max_depth),
        'max_leaves': trial.suggest_int('max_leaves', min_max_leaf_nodes, max_max_leaf_nodes),
        'n_estimators': trial.suggest_int('n_estimators', min_n_estimators, max_n_estimators)
    }

    gb_xgb_opt = xgb.XGBRegressor(param=param, 
                                  objective='reg:squarederror',
                                  verbosity=verbose,
                                  random_state=random_state
                                 )

    gb_xgb_opt = gb_xgb_opt.fit(x_train, y_train)
    y_val_pred = gb_xgb_opt.predict(x_val)
    
    return math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))


gb_optuna = optuna.create_study(direction='minimize')
start_time = time.time()
gb_optuna.optimize(xgradboosting_objective, n_trials=budget)
end_time = time.time()

summary['gradient_boosting'] = summary['gradient_boosting'].append(pd.Series({
    'Time (sec)': '{:.4f}'.format(end_time - start_time), 
    'Score (RMSE)': gb_optuna.best_value,
    'Learning rate': gb_optuna.best_params['eta'],
    'Max. depth': gb_optuna.best_params['max_depth'],
    'Max. leaf nodes': gb_optuna.best_params['max_leaves'],
    'Gamma (min_split_loss)': gb_optuna.best_params['gamma'],
    'Lambda': gb_optuna.best_params['lambda'],
    'Alpha': gb_optuna.best_params['alpha'],
    'N. estimators': gb_optuna.best_params['n_estimators']  
    },
    name='optuna_xgboost'))

[I 2021-01-13 15:25:21,347] A new study created in memory with name: no-name-3f72ac32-38db-40a7-9172-274b9088f72a
[I 2021-01-13 15:25:30,302] Trial 0 finished with value: 409.80286980982993 and parameters: {'lambda': 0.3207145560099033, 'alpha': 0.12901374278798036, 'gamma': 0.9298885956847187, 'eta': 0.11878362557748356, 'max_depth': 19, 'max_leaves': 20, 'n_estimators': 87}. Best is trial 0 with value: 409.80286980982993.
[I 2021-01-13 15:25:38,446] Trial 1 finished with value: 409.80286980982993 and parameters: {'lambda': 0.39229298850004907, 'alpha': 0.29383473771735485, 'gamma': 0.5275541655847883, 'eta': 0.12001232732053646, 'max_depth': 24, 'max_leaves': 15, 'n_estimators': 132}. Best is trial 0 with value: 409.80286980982993.
[I 2021-01-13 15:25:48,225] Trial 2 finished with value: 409.80286980982993 and parameters: {'lambda': 0.14139686492911407, 'alpha': 0.42492610165887745, 'gamma': 0.5103158692669374, 'eta': 0.7319981991696786, 'max_depth': 16, 'max_leaves': 2, 'n_estimator

In [21]:
summary['knn']

,Time (sec),Score (RMSE),N. neighbors,Weights,P
default,0.1176,455.123868,5,uniform,2
optuna,33.7278,424.954880,11,distance,1


In [48]:
summary['random_forest']

,Time (sec),Score (RMSE),Min. samples split,Criterion,Max. depth,N. estimators,Max. features
default,82.5335,375.560721,2,mse,None,100,1
optuna,233.6654,374.129312,0.00872,mse,None,171,0.667976
optuna,1031.8257,373.977008,0.00742,mae,NaN,101,sqrt


In [31]:
summary['gradient_boosting']

,Time (sec),Score (RMSE),Criterion,Learning rate,Max. depth,Max. leaf nodes,Min. samples leaf,Min. samples split,N. estimators,Alpha,Gamma (min_split_loss),Lambda
default,48.0549,389.223359,friedman_mse,0.100000,3.0,None,1.0,2.0,100.0,NaN,NaN,NaN
default_xgboost,8.5909,409.802870,NaN,0.300000,6.0,0,NaN,NaN,100.0,0.000000,0.000000,1.000000
optuna_xgboost,199.0509,409.802870,NaN,0.118784,19.0,20,NaN,NaN,87.0,0.129014,0.929889,0.320715


In [32]:
#Dummy regressor(mean)
math.sqrt(metrics.mean_squared_error(y_val, [y_val.mean() for i in range(len(y_val))]))

666.6691142412727

# 2. ATTRIBUTE SELECTION

## 2.1 Select from all attributes

**Are all 550 input attributes actually necessary in order to get a good model? Is it possible to have an accurate model that uses fewer than 550 variables? How many?**

For this question we will be using the best model we had in previous section and now include the parameter for select only certain attributes.

In [12]:
min_max_depth = 2
max_max_depth = 15
min_n_estimators = 50
max_n_estimators = 200
min_n_k = 10
max_n_k = 30

In [13]:
np.random.seed(random_state)
def random_forest_objective_attr(trial):
    k = trial.suggest_int('k', min_n_k, max_n_k)
    min_samples_split = trial.suggest_uniform('min_samples_split', 0+sys.float_info.min, 1)
    criterion = trial.suggest_categorical('criterion', ['mse','mae'])
    max_depth = trial.suggest_int('max_depth', min_max_depth, max_max_depth, log=True)
    n_estimators = trial.suggest_int('n_estimators', min_n_estimators, max_n_estimators)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])

    clf = Pipeline([
      ('feature_selection', feature_selection.SelectKBest(feature_selection.f_regression, k=k)),
      ('regression', ensemble.RandomForestRegressor(
          random_state=random_state,
          min_samples_split=min_samples_split,
          criterion=criterion,
          max_depth=max_depth,
          n_estimators=n_estimators,
          max_features=max_features
      ))
    ])

    clf = clf.fit(x_train, y_train)
    y_val_pred = clf.predict(x_val)
    return math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))

rf_attr_optuna = optuna.create_study(direction='minimize')
start_time = time.time()
rf_attr_optuna.optimize(random_forest_objective_attr, n_trials=budget, n_jobs=n_jobs)
end_time = time.time()
print(end_time-start_time)

[I 2021-01-13 13:59:52,067] A new study created in memory with name: no-name-f677b493-b3e6-48bb-87d6-43f34315514d
[I 2021-01-13 14:00:57,538] Trial 0 finished with value: 444.03711882821585 and parameters: {'k': 59, 'min_samples_split': 0.31820927889533557, 'criterion': 'mae', 'max_depth': 15, 'n_estimators': 77, 'max_features': 'auto'}. Best is trial 0 with value: 444.03711882821585.
[I 2021-01-13 14:00:58,050] Trial 1 finished with value: 719.5022353639617 and parameters: {'k': 51, 'min_samples_split': 0.6987448939434953, 'criterion': 'mae', 'max_depth': 4, 'n_estimators': 176, 'max_features': 'auto'}. Best is trial 0 with value: 444.03711882821585.
[I 2021-01-13 14:00:58,225] Trial 2 finished with value: 668.4403422905759 and parameters: {'k': 188, 'min_samples_split': 0.8572131246824229, 'criterion': 'mse', 'max_depth': 2, 'n_estimators': 106, 'max_features': 'auto'}. Best is trial 0 with value: 444.03711882821585.
[I 2021-01-13 14:03:25,676] Trial 3 finished with value: 504.586309

587.7053737640381


In [14]:
#TODO: Conclusions
print(rf_attr_optuna.best_params, rf_attr_optuna.best_value)

{'k': 273, 'min_samples_split': 0.3102503141152435, 'criterion': 'mae', 'max_depth': 9, 'n_estimators': 145, 'max_features': 'sqrt'} 413.210766994515


## 2.2 Use only Sotavento attributes
**Is it enough to use only the attributes for the actual Sotavento location? (13th location in the grid)**

We will select only Sotavento attributes and use the best model in previous section to train a model.

In [15]:
sot_attr = []
for attr in x_train.columns:
    if int(attr.split('.')[-1]) == 13:
        sot_attr.append(attr)

x_train_sot = x_train[sot_attr]
x_val_sot = x_val[sot_attr]
x_test_sot = x_test[sot_attr]
print(x_train_sot.shape,x_val_sot.shape,x_test_sot.shape)

(2528, 22) (1299, 22) (2110, 22)


In [17]:
np.random.seed(random_state)
def random_forest_sot_objective(trial):
    min_samples_split = trial.suggest_uniform('min_samples_split', 0+sys.float_info.min, 1)
    criterion = trial.suggest_categorical('criterion', ['mse','mae'])
    max_depth = trial.suggest_int('max_depth', min_max_depth, max_max_depth)
    n_estimators = trial.suggest_int('n_estimators', min_n_estimators, max_n_estimators)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])

    clf = ensemble.RandomForestRegressor(
        random_state=random_state,
        min_samples_split=min_samples_split,
        criterion=criterion,
        max_depth=max_depth,
        n_estimators=n_estimators,
        max_features=max_features
        )

    clf = clf.fit(x_train_sot, y_train)
    y_val_pred = clf.predict(x_val_sot)
    return math.sqrt(metrics.mean_squared_error(y_val, y_val_pred))

rf_sot_optuna = optuna.create_study(direction='minimize')
start_time = time.time()
rf_sot_optuna.optimize(random_forest_sot_objective, n_trials=budget, n_jobs=n_jobs)
end_time = time.time()

[I 2021-01-13 13:41:40,158] A new study created in memory with name: no-name-47d55393-09de-4246-a862-7ba94ba8ee42
[I 2021-01-13 13:41:40,344] Trial 1 finished with value: 668.3057723890951 and parameters: {'min_samples_split': 0.7773231353617598, 'criterion': 'mse', 'max_depth': 16, 'n_estimators': 81, 'max_features': 'sqrt'}. Best is trial 1 with value: 668.3057723890951.
[I 2021-01-13 13:41:41,298] Trial 0 finished with value: 415.3216110107275 and parameters: {'min_samples_split': 0.20071034246198016, 'criterion': 'mse', 'max_depth': 22, 'n_estimators': 157, 'max_features': 'log2'}. Best is trial 0 with value: 415.3216110107275.
[I 2021-01-13 13:41:41,419] Trial 4 finished with value: 668.3204020271736 and parameters: {'min_samples_split': 0.7537659986365126, 'criterion': 'mse', 'max_depth': 8, 'n_estimators': 108, 'max_features': 'log2'}. Best is trial 0 with value: 415.3216110107275.
[I 2021-01-13 13:41:42,324] Trial 2 finished with value: 394.79061560842706 and parameters: {'min_

KeyboardInterrupt: 

In [ ]:
#TODO: Conclusions
print(rf_sot_optuna.best_params, rf_sot_optuna.best_value)